In [1]:
import readfof
import numpy as np

from scipy.interpolate import interp1d
from scipy import integrate
import matplotlib.pyplot as plt#约定俗成的写法plt

from scipy.optimize import curve_fit
from scipy import special

from scipy.integrate import odeint,quad,dblquad,tplquad,simps
import scipy.special as special
from numpy.polynomial import polynomial, legendre
import sympy as sym
from colossus.cosmology import cosmology


from mcfit import xi2P
from scipy import linalg

In [2]:
##读取暗物质功率谱 
params = {'flat': True, 'H0': 67.72, 'Om0': 0.315, 'Ob0': 0.049, 'sigma8': 0.82, 'ns': 0.95}
cosmology.addCosmology('myCosmo', **params)
cosmo = cosmology.setCosmology('myCosmo')
k=10**np.linspace(-4,0.5,10000)
Pnl0 = cosmo.matterPowerSpectrum(k,z=0)
Pdm=interp1d(k,Pnl0,kind='cubic')

In [4]:
Omega_k = 0.
Omega_r = 0
Omega_m = 0.3175
Omega_b=0.049
Omega_cdm=Omega_m-Omega_b
Omega_L = 1-Omega_k - Omega_r - Omega_m
w0 = -1.
wa = 0.
h=0.6772
H0=67.72
speed_of_light = 2.998e5 ##speed of light in km/s
H0Mpc = H0/speed_of_light # H0*Mpc/c 

## rho_DE/rho_{DE0} as a function of a
def rhoDE_ratio(a):
    return np.exp(-3.*((1.+w0+wa)*np.log(a)+wa*(1.-a)))

def Hofa(a):
    return 100*h*np.sqrt(Omega_r/a**4 + Omega_m/a**3 + Omega_k/a**2 + Omega_L)

def Hofz(z):
    return 100*h*np.sqrt(Omega_L + Omega_m*(1+z)**3 + Omega_k * (1+z)**2 + Omega_r*(1+z)**4)




#H^2 as a function of a, in unit of H_0^2
def Hsqofa(a):
    return Omega_r/a**4 + Omega_m/a**3 + Omega_k/a**2 + Omega_L*rhoDE_ratio(a)

#dH/dt as a function of a, in unit of H_0^2
def dotHofa(a):
    return -2*Omega_r/a**4 - 1.5*Omega_m/a**3 - Omega_k/a**2 - 1.5*(1.+w0+wa*(1-a))*Omega_L * rhoDE_ratio(a)


def growth_eq(y, lna):  #y = [G , dG/dN]
    a = np.exp(lna)
    Hsq = Hsqofa(a)
    dotH = dotHofa(a)
    return np.array( [ y[1], -(4+dotH/Hsq)*y[1]-(3+dotH/Hsq-1.5/Hsq/a**3*Omega_m)*y[0] ] )


#set initial conditions
a_ini = 1.e-2 #it can be arbitrary, as long as 3.e-4 << a_ini << 1
Hsq_ini = Hsqofa(a_ini)
dotH_ini = dotHofa(a_ini)
coef_1 = 4+dotH_ini/Hsq_ini
coef_0 = 3+dotH_ini/Hsq_ini-1.5/Hsq_ini/a_ini**3*Omega_m
eps = coef_0/(coef_1 - coef_0 - 1.)
G_ini = 1.+ eps #this is arbitrary normalization; will renormalize in the end
dGdN_ini = -eps

nsteps = 500
lna = np.linspace(np.log(a_ini), 0., nsteps)
a = np.exp(lna)
Dbya = odeint(growth_eq, [ G_ini, dGdN_ini ], lna)
#now renormalize to D(z=0) = 1
D = Dbya[:, 0] / Dbya[nsteps-1, 0] * a
#compute the growth rate f = d ln D/d ln a
f = 1. + Dbya[:,1]/Dbya[:,0]

#interpolate to get functions D(a), f(a)
Dofa = interp1d(a, D, kind="cubic")
fofa = interp1d(a, f, kind="cubic")

def fofz(z):
    return fofa(1/(1+z))


a_test = 0.5
print("D(", a_test, ") = ", Dofa(a_test))
print("f(", a_test, ") = ", fofa(a_test))


#make plot
#plt.plot(lna, Dbya[:,0], '-', lna, f, '--')
#plt.xlabel(r'$\ln a$')
#plt.legend([r'$D/a$ (not normalized)', r'$f$'], loc='best')
#plt.how()

D( 0.5 ) =  0.6059382403253776
f( 0.5 ) =  0.8778910695109503


In [9]:
##steps of simpson intergrate
nsimps=40
nsimps1=40



## calculate sigma of cylinder from Pk
def cld_th_fast(R,H,f,b,sigmav):
    R_cylinder=R
    bg=b
    growthfactor=f
    h=H
    beta=growthfactor/bg
    sigmav_of_H=sigmav/100


    def r_max(kz):
        return 1
    def r_min(kz):
        return 1e-3

    ##if use dblquad
    #return np.sqrt(2*dblquad(P_of_kv_kz,1e-4,1,r_min,r_max,epsabs=1e-1)[0])

    ##if use simps
    klist=np.logspace(-4,0,nsimps1)
    klist1=np.logspace(-4,0,nsimps1)

    
    def f1(kz):
        def f2(kv):
            kmode=np.sqrt(kv**2+kz**2)
            mnu=kz/kmode
            P_rsd=(bg**2)*Pdm(kmode)*((1+beta*1*mnu**2)**2)/(1+(kmode**2)*(mnu**2)*(sigmav_of_H**2))
            return P_rsd*((2*special.jv(1,kv*R_cylinder)/(kv*R_cylinder))**2)*kv


        return simps(f2(klist1),klist1)*((np.sin(kz*h/2)/(kz*h/2))**2)
    



    #return np.sqrt(2*simps(f1(klist),klist)*1/(4*np.pi**2))
    #return f1(0.1)
    #return np.sqrt(2*quad(f1,1e-4,1,epsabs=1e-1)[0]*1/(4*np.pi**2))
    inner_simpsx=np.logspace(-4,0,nsimps1)
    inner_simpsy=np.zeros(nsimps1)
    for i in range(nsimps1):
        inner_simpsy[i]=f1(inner_simpsx[i])
    return np.sqrt(2*simps(inner_simpsy,inner_simpsx)*1/(4*np.pi**2))